## Bacterial growth curves

This notebook reproduces the analysis to create bacterial growth curves. Images are first aligned and segmented. Then the cells are tracked from frame to frame using the linear assignment problem approach.

First we have to import packages

In [7]:
import h5py
import tifffile as tiff
from keras.backend.common import _UID_PREFIXES

from cnn_functions import nikon_getfiles, get_image, run_models_on_directory, get_image_sizes, segment_nuclei, segment_cytoplasm, dice_jaccard_indices
from model_zoo import sparse_bn_feature_net_61x61 as cyto_fn
from model_zoo import sparse_bn_feature_net_61x61 as nuclear_fn

import os
import numpy as np

from cnn_functions import create_masks, get_image, align_images, crop_images, make_tracks, get_lineage
from cnn_functions import plot_lineage, plot_lineage_numbers, plot_lineage_total
import matplotlib as mpl
import scipy
mpl.rcParams['pdf.fonttype'] = 42


ImportError: cannot import name create_masks